# 24. 고급 분석과 머신러닝

## 24.3 고수준 MLlib의 개념

### 저수준 데이터 타입

In [1]:
from pyspark.ml.linalg import Vectors

densVec = Vectors.dense(1.0, 2.0, 3.0)
size = 3
idx = [1, 2]
values = [2.0, 3.0]
sparseVec = Vectors.sparse(size, idx, values)

In [2]:
densVec

DenseVector([1.0, 2.0, 3.0])

In [3]:
sparseVec

SparseVector(3, {1: 2.0, 2: 3.0})

## 24.4 MLlib 실제로 사용하기
+ 범주형 레이블 1, 범주형 변수 1, 수치형 변수 2 데이터로 실습

In [4]:
# 세션 생성
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Machine Learning examples") \
    .getOrCreate()

spark.conf.set('spark.sql.shuffle.partitions', 5)

In [5]:
df = spark.read.json("../BookSamples/data/simple-ml/")
df.orderBy("value2").show()

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
|green| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     2|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|  red| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|green|good|     1|14.386294994851129|
|green|good|    12|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|  red|good|    35|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



### 24.4.1 변환자를 사용해서 피처 엔지니어링 수행
+ 모든 입력변수는 Double, Vector[Double] 타입으로 구성 필요
+ RFormula를 사용하여 쉽게 변환 가능

In [6]:
df.select("lab").distinct().show()

+----+
| lab|
+----+
| bad|
|good|
+----+



In [7]:
from pyspark.ml.feature import RFormula

supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")
fittedRF = supervised.fit(df)
preparedDF = fittedRF.transform(df)
preparedDF.show(20, False)

+-----+----+------+------------------+----------------------------------------------------------------------+-----+
|color|lab |value1|value2            |features                                                              |label|
+-----+----+------+------------------+----------------------------------------------------------------------+-----+
|green|good|1     |14.386294994851129|(10,[1,2,3,5,8],[1.0,1.0,14.386294994851129,1.0,14.386294994851129])  |1.0  |
|blue |bad |8     |14.386294994851129|(10,[2,3,6,9],[8.0,14.386294994851129,8.0,14.386294994851129])        |0.0  |
|blue |bad |12    |14.386294994851129|(10,[2,3,6,9],[12.0,14.386294994851129,12.0,14.386294994851129])      |0.0  |
|green|good|15    |38.97187133755819 |(10,[1,2,3,5,8],[1.0,15.0,38.97187133755819,15.0,38.97187133755819])  |1.0  |
|green|good|12    |14.386294994851129|(10,[1,2,3,5,8],[1.0,12.0,14.386294994851129,12.0,14.386294994851129])|1.0  |
|green|bad |16    |14.386294994851129|(10,[1,2,3,5,8],[1.0,16.0,14.38629

In [8]:
preparedDF.printSchema()

root
 |-- color: string (nullable = true)
 |-- lab: string (nullable = true)
 |-- value1: long (nullable = true)
 |-- value2: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)



+ 데이터의 임의분할

In [9]:
train, test = preparedDF.randomSplit([0.7, 0.3])

### 24.4.2 추정자

In [10]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol='label', featuresCol='features')

In [11]:
# 옵션 검토
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

In [12]:
fittedLR = lr.fit(train)

In [13]:
fittedLR.transform(test).select("*").show(20, False)

+-----+----+------+------------------+----------------------------------------------------------------------+-----+----------------------------------------+-------------------------------------------+----------+
|color|lab |value1|value2            |features                                                              |label|rawPrediction                           |probability                                |prediction|
+-----+----+------+------------------+----------------------------------------------------------------------+-----+----------------------------------------+-------------------------------------------+----------+
|blue |bad |8     |14.386294994851129|(10,[2,3,6,9],[8.0,14.386294994851129,8.0,14.386294994851129])        |0.0  |[131.22693647979438,-131.22693647979438]|[1.0,1.0206236079946356E-57]               |0.0       |
|blue |bad |12    |14.386294994851129|(10,[2,3,6,9],[12.0,14.386294994851129,12.0,14.386294994851129])      |0.0  |[157.5848005712356,-157.5848005712356

### 24.4.3 워크플로를 파이프라인으로 만들기

+ 변환자 객체나 모델 객체가 다른 파이프라인에서 재사용되지 않도록 함

In [14]:
from pyspark.ml import Pipeline

train, test = df.randomSplit([0.7, 0.3])
rForm = RFormula()
lr = LogisticRegression().setLabelCol("label").setFeaturesCol("features")

stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

### 24.4.4 모델 학습 및 평가

+ 다양한 하이퍼파라미터 테스트 사례
    + 두 개 버전의 RFormual
    + 세 개 다른 옵션의 ElasticNet 파라미터
    + 두 개 서로 다른 옵션의 일반화 파라미터

In [15]:
from pyspark.ml.tuning import ParamGridBuilder

params = ParamGridBuilder()\
    .addGrid(rForm.formula, [
        "lab ~ . + color:value1",
        "lab ~ . + color:value1 + color:value2"])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .addGrid(lr.regParam, [0.1, 0.2])\
    .build()

In [16]:
# 평가

from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()\
    .setMetricName("areaUnderROC")\
    .setRawPredictionCol("prediction")\
    .setLabelCol("label")

In [17]:
# 튜닝

from pyspark.ml.tuning import TrainValidationSplit

tvs = TrainValidationSplit()\
    .setTrainRatio(0.75)\
    .setEstimatorParamMaps(params)\
    .setEstimator(pipeline)\
    .setEvaluator(evaluator)

In [18]:
# 드디어 학습

tvsFitted = tvs.fit(train)

In [19]:
# 최종 예측 평가

tvsFitted.transform(test).show()
evaluator.evaluate(tvsFitted.transform(test))

+-----+----+------+------------------+--------------------+-----+--------------------+--------------------+----------+
|color| lab|value1|            value2|            features|label|       rawPrediction|         probability|prediction|
+-----+----+------+------------------+--------------------+-----+--------------------+--------------------+----------+
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|[1.16065512002940...|[0.76145173318613...|       0.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|[1.16065512002940...|[0.76145173318613...|       0.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|[1.16065512002940...|[0.76145173318613...|       0.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|[1.16065512002940...|[0.76145173318613...|       0.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|[1.16065512002940...|[0.76145173318613...|       0.0|
| blue| bad|    12|14.386294994851129|(10,[2,3,6

0.75

In [20]:
print(tvsFitted.validationMetrics)

[0.8375, 0.8375, 0.8375, 0.7875, 0.8375, 0.75, 0.8375, 0.8375, 0.7875, 0.85, 0.85, 0.7]


In [23]:
print(tvsFitted.explainParams())

estimator: estimator to be cross-validated (current: Pipeline_243c46cf422e)
estimatorParamMaps: estimator param maps (current: [{Param(parent='RFormula_cb403a9f5591', name='formula', doc='R model formula'): 'lab ~ . + color:value1', Param(parent='LogisticRegression_95f84505e634', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_95f84505e634', name='regParam', doc='regularization parameter (>= 0).'): 0.1}, {Param(parent='RFormula_cb403a9f5591', name='formula', doc='R model formula'): 'lab ~ . + color:value1', Param(parent='LogisticRegression_95f84505e634', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_95f84505e634', name='regParam', doc='regularization parameter (>= 0).'): 0.2}, {Param

In [24]:
evaluator.explainParams()

'labelCol: label column name. (default: label, current: label)\nmetricName: metric name in evaluation (areaUnderROC|areaUnderPR) (default: areaUnderROC, current: areaUnderROC)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction, current: prediction)'

### 24.4.5 모델 저장 및 적용

+ 특정 알고리즘에 대한 '모델' 버전을 사용하여 디스크에 저장된 모델을 불러와야 함
    + CrossValidator는 CrossValidatorModel이 저장된 버전을 읽어야 하는 식

In [25]:
tvsFitted.bestModel.write().overwrite().save('./model')

In [26]:
from pyspark.ml import PipelineModel

bestModel = PipelineModel.load('./model')

In [27]:
evaluator.evaluate(bestModel.transform(test)) # 결과 동일

0.75